In [11]:
print ("hola")

hola


In [2]:
#Se importa los modulos necesarios a utilizar

import pandas as pd
import os
import glob
import datetime as dt
from time import strftime
import numpy as np

In [12]:

# En base a la escasa información que se tiene sobre el proceso de extracción de los datos
# y dado que se presentan en multiples formatos e inconsistencias, supuse que a la hora de
# continuar con la extracción de futuros datos se realizaría por los mismos medios, generando
# archivos con formatos e inconsistencias similares, por lo que lo mas adecuado teniendo en cuenta
# el tiempo disponible fue realizar una unica función capaz de abrir todos los archivos contenidos
# en un directorio, sortearlos según su formato y aplicarle las transformaciones necesarias para
# normalizar los datos y el formato; y ademas intentar corregir la mayor cantidad de inconsistencias.


# En el futuro y con algo mas de tiempo es posible separar la función en multiples mas especificas para abarcar mas variaciones
def file_format(filepath):
    
    # Separa el archivo en nombre y formato 
    file = os.path.basename(filepath)
    split_tup = os.path.splitext(file)
    file_name = split_tup[0]
    file_extension = split_tup[1]
    
    #XLSX (multiples hojas) a CSV (CSV por hoja)
    if file_extension == ".xlsx":
        
        # For loop por cada Hoja del XLSX
        for sheet_name, df in pd.read_excel(filepath, sheet_name=None, dtype= {"precio":str}).items():  
            
            #Normalización del orden de columnas
            col = df.pop("producto_id")
            df.insert(1, "producto_id", col)  
            
            # Transformar la columna "sucursal_id", ya que algunos datos se encuentran como Datetime y están rotados
            # Se usa Try/Except ya que no todas las hojas presentan estas inconsistencias.
            try:
                cont = -1
                df["nuevo"] = ""
                for a in df["sucursal_id"]:  #Inicialmente se intentó usar una unica condición en la serie, solo se logró con un for loop.
                    cont+=1
                    if type(a) == dt.datetime:
                        df["nuevo"].iloc[cont] = df["sucursal_id"].iloc[cont].strftime("%d-%m-%Y")  # strftime rota el dato y transforma a string
                df2 = df["nuevo"].str.split("-", expand=True)                                       # Se guarda en una columna "nueva" para no interferir
                df2 =df2.replace("",None)                                                           # con el for loop
                df2 = df2.fillna(value=0).astype(int) # Transformar a INT para eliminar Zeros iniciales        
                df["sucursal_id"][df2[2] != 0] = df2[[0,1,2]].astype(str).apply(lambda x: '-'.join(x), axis=1)
            except:
                pass
            del df["nuevo"]
            
            # Se eliminan guiones y decimales no deseados en columna "producto_id"
            # Completar las 13 cifras del codigo EAN con zeros iniciales 
            df.loc[df['producto_id'].notnull(), 'producto_id'] = df.loc[df['producto_id'].notnull(), 'producto_id'].astype(str).str.split("-").str[-1]
            df["producto_id"] = df["producto_id"].str.split(".").str[0]  
            df["producto_id"] = df["producto_id"].str.zfill(13)
            
            # Eliminar duplicados
            df = df.drop_duplicates()
            
            # Guardar el archivo en formato .CSV
            df.to_csv(f"{sheet_name}.csv", encoding="utf-8", index=False)
        
    #JSON a CSV
    elif file_extension == ".json":   
        df = pd.read_json(filepath)
        df["producto_id"] = df["producto_id"].astype(str)
        df["producto_id"] = df['producto_id'].str.split('-').str[-1] # Se eliminan guiones y decimales no deseados en columna "producto_id"
        df["producto_id"] = df["producto_id"].str.zfill(13)          # Completar las 13 cifras del codigo EAN con zeros iniciales 
        df = df.drop_duplicates()                                    # Eliminar duplicados
        df.to_csv(file_name + ".csv", encoding ="utf-8", index=False)# Guardar el archivo en formato .CSV

    #TXT a CSV. Se realiza la misma transformación realizada para el JSON
    elif file_extension == ".txt":
        df = pd.read_csv(filepath, delimiter="|")
        df["producto_id"] = df['producto_id'].str.split('-').str[-1]
        df["producto_id"] = df["producto_id"].str.zfill(13)
        df = df.drop_duplicates()
        df.to_csv(file_name + ".csv", encoding="utf-8", index=False)
    
    #PARQUET a CSV. Se realiza la misma transformación realizada para el JSON   
    elif file_extension == ".parquet":
        df = pd.read_parquet(filepath)
        df["id"] = df['id'].str.split('-').str[-1]
        df["id"] = df["id"].str.zfill(13)
        df = df.drop_duplicates(subset="id")
        df.to_csv(file_name + ".csv", encoding= "utf-8", index=False)
    
    # CSV a CSV. Ya que vienen con encodings diferentes se realiza la lectura 2 veces 
    # Se realiza la misma transformación realizada para el JSON     
    elif file_extension == ".csv":
        encodings = ["utf-8", "utf-16"]
        for encode in encodings:
            try:
                df = pd.read_csv(filepath, encoding=encode)
                try:
                    df["producto_id"] = df['producto_id'].str.split('-').str[-1]
                    df["producto_id"] = df["producto_id"].str.zfill(13)
                except:
                    pass  
                df = df.drop_duplicates()             
                df.to_csv(file_name + ".csv", encoding="utf8", index=False)
            except:
                pass      
        
    #formato no reconocido, se evitan errores por formatos no añadidos    
    else:
        print (file_name+file_extension, "es nuevo formato aun no añadido a la función")
        

In [10]:
#Codigo que abre todos los archivos del directorio y ejecuta en cada uno la función previa
path = ("D:\Henry\Repos\Labs PI\PI01_DATA_ENGINEERING-main\Datasets")
for filename in glob.glob(os.path.join(path, '*.*')):
   with open(os.path.join(os.getcwd(), filename), 'r') as f:
       file_format (filename)


# Al final del proceso se obtienen archivos .CSV en el directorio donde se encuentra el jupiter con el codigo
# Todos los archivos se encuentran normalizados y con gran mayoria de correcciones listos para ser cargados
# a la base de datos deseada

README.md es nuevo formato aun no añadido a la función
